# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [1]:
# Reinstalling torch with the right CUDA bindings.
!pip3 install -U https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install -U https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

  Using cached https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0
  Using cached https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torchvision 0.3.0
    Uninstalling torchvision-0.3.0:
      Successfully uninstalled torchvision-0.3.0


In [0]:
# Basic imports.
import os
import numpy as np
import torch

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [11]:
# Setting predefined arguments.
args = {
    'epoch_num': 20,      # Number of epochs.
    'n_classes': 10,      # Number of classes.
    'lr': 0.05,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 500,    # Mini-batch size.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


# O pacote Torchvision

Assim como no MXNet, o subpacote [datasets](https://pytorch.org/docs/stable/torchvision/datasets.html) do [torchvision](https://pytorch.org/docs/stable/torchvision/index.html) do Pytorch possui dataloaders para vários datasets conhecidos da literatura de Visão Computacional. O CIFAR10, já usado previamente no curso, é um desses dataset clássicos de classificação que possuem implementações que podem ser carregados por meio de uma única linha de código. Outros datasets clássicos presentes no torchvision são: MNIST (classificação), Fasion-MNIST (classificação), ImageNet (classificação), VOC (segmentação e detecção), COCO (detecção e captioning), etc. No bloco de código abaixo são carregados os datasets de treino e teste do CIFAR10 e seus respectivos dataloaders.

O torchvision também possui outras funções de apoio ao treino de modelos para Deep Learning em imagens, como a definição de modelos (pré-treinados ou não) no subpacote [models](https://pytorch.org/docs/stable/torchvision/models.html) e a possibilidade de realizar pré-processamentos em imagens (incluindo Data Augmentation) por meio de transformações comuns em imagens no subpacote [transforms](https://pytorch.org/docs/stable/torchvision/transforms.html). Essas transformações também são demonstradas no código abaixo.

In [12]:
# Root directory for the dataset (to be downloaded).
root = './'

data_transforms = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor()
])

# Setting datasets and dataloaders.
train_set = datasets.MNIST(root,
                           train=True,
                           download=True,
                           transform=data_transforms)
test_set = datasets.MNIST(root,
                          train=False,
                          download=False,
                          transform=data_transforms)

train_loader = DataLoader(train_set,
                          args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         args['batch_size'],
                         num_workers=args['num_workers'],
                         shuffle=False)

# Printing training and testing dataset sizes.
print('Size of training set: ' + str(len(train_set)) + ' samples')
print('Size of test set: ' + str(len(test_set)) + ' samples')

Size of training set: 60000 samples
Size of test set: 10000 samples


# Programação Orientada a Objetos

Faz-se necessário o uso de  [Programação Orientada a Objetos](https://www.tutorialspoint.com/python/python_classes_objects.htm) (POO) à medida que se começa a lidar com tarefas mais específicas tratadas por algoritmos de Deep Learning. Algumas vantagens que POO trazem para a programação de algoritmos de Deep Learning são:

1.   Organização do código. Criar uma arquitetura com centenas de camadas podendo conter várias *skip connections* no meio de um bloco de código, como vínhamos fazendo anteriormente, torna o nosso código desorganizado;
2.   Reaproveitamento dos códigos básicos do framework (i.e. dataloaders, módulos/camadas de redes predefinidos, funções de perda, etc) para adaptá-los para novas tarefas minimizando a quantidade de código que necessita ser escrita;
3.   Códigos escritos em formato POO podem ser reutilizados em outros pontos da aplicação. Arquiteturas como [DenseNets](https://arxiv.org/abs/1608.06993), [ResNets](https://arxiv.org/abs/1512.03385) ou [U-Nets](https://arxiv.org/abs/1505.04597) possuem blocos de código que se repetem, os quais podem ser simplificados com o uso de POO;
4.   POO também permite uma melhor modularização do código, de forma que mudanças (i.e. loss function, arquitetura, dataloader, etc) possam ser feitas em módulos isolados sem afetar o resto do código.

No caso da implementação de modelos usando classes no Pytorch, normalmente é preciso se preocupar apenas com a implementação de dois métodos: o *\_\_init\_\_()* e o *forward()*. O *\_\_init\_\_()* é conhecido como o construtor da classe e define a arquitetura da rede neural que é implementada pela classe, ou seja, é nesse método que se definem as camadas da rede neural. Já o *forward()* serve para definir o fluxo de dados que passará pelas camadas definidas no *\_\_init\_\_()*. O *forward()* recebe sempre como entrada os dados que passarão pelo modelo e deve retornar o output do modelo após processar esses dados de entrada ao longo de suas camadas. É notável que entre as camadas convolucionais e as camadas Fully Connected, faz-se necessário chamar o método [*view()*](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) para linearizar os dados do tensor.

Também é preciso lembrar de sempre definir a classe em que o modelo está sendo implementado como subclasse de *nn.Module*, como mostrado abaixo.

In [13]:
# LeNet implementation.
class LeNet(nn.Module):
    
    def __init__(self, num_classes=10):

        super(LeNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=0, ),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(6, 16, kernel_size=5, padding=0),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(16, 120, kernel_size=5, padding=0),
            nn.Tanh(),
            
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84, bias=True),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=num_classes, bias=True)
        )       
        self.initialize_weights()
    
    # Function for randomly initializing weights.
    def initialize_weights(self):
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        
        x = self.features(x)
        x = x.view(x.size(0), -1) # Linearizing.
        x = self.classifier(x)
        return x

# Instantiating architecture.
net = LeNet(args['n_classes']).to(args['device'])

# Printing architecture.
print(net)

LeNet(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


# Definindo o otimizador

O Pytorch possui vários otimizadores prontos no subpacote [optim](https://pytorch.org/docs/stable/optim.html), desde o SGD básico a otimizadores mais complexos e com taxas de aprendizado por parâmetro como o Adagrad, RMSProp e Adam.

In [0]:
optimizer = optim.SGD(net.parameters(),
                      lr=args['lr'],
                      momentum=args['momentum'],
                      weight_decay=args['weight_decay'])

for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(args['device'])

# Definindo a loss

O subpacote [nn](https://pytorch.org/docs/stable/nn.html) possui várias funções de perda para diferentes tarefas (i.e. Cross Entropy, Negative Log Likelihood, loss L1, MSE, Kullback Leibler Divergence, etc) implementadas por padrão.



In [0]:
criterion = nn.CrossEntropyLoss().to(args['device'])

# Criando funções para Treino e Teste

Iterando sobre os datasets/dataloaders de treino e teste do CIFAR10. Abaixo são implementadas a função *train()* que itera sobre os batches do dataset de treino e atualiza o modelo e a função *test()* que apenas realiza o forward dos dados de teste no modelo e calcula a acurácia no dataset de teste para o modelo no estado atual.

In [0]:
def train(train_loader, net, criterion, optimizer, epoch):

    # Setting network for training mode.
    net.train()

    # Lists for losses and metrics.
    train_loss = []
    
    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images, labels and paths for batch.
        inps, labs = batch_data
        
        # Casting to cuda variables.
        inps = inps.to(args['device'])
        labs = labs.to(args['device'])
        
        # Clears the gradients of optimizer.
        optimizer.zero_grad()

        # Forwarding.
        outs = net(inps)

        # Computing loss.
        loss = criterion(outs, labs)

        # Computing backpropagation.
        loss.backward()
        optimizer.step()
        
        # Updating lists.
        train_loss.append(loss.data.item())
    
    train_loss = np.asarray(train_loss)
    
    # Printing training epoch loss and metrics.
    print('--------------------------------------------------------------------')
    print('[epoch %d], [train loss %.4f +/- %.4f]' % (
        epoch, train_loss.mean(), train_loss.std()))
    print('--------------------------------------------------------------------')
    
def test(test_loader, net, criterion, epoch):

    # Setting network for evaluation mode.
    net.eval()

    # Lists for losses and metrics.
    test_loss = []
    prd_list = []
    lab_list = []
    
    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images, labels and paths for batch.
        inps, labs = batch_data

        # Casting to cuda variables.
        inps = inps.to(args['device'])
        labs = labs.to(args['device'])

        # Forwarding.
        outs = net(inps)

        # Computing loss.
        loss = criterion(outs, labs)
        
        # Obtaining predictions.
        prds = outs.data.max(dim=1)[1].cpu().numpy()
        
        # Updating lists.
        test_loss.append(loss.data.item())
        prd_list.append(prds)
        lab_list.append(labs.detach().cpu().numpy())
    
    # Computing accuracy.
    acc = metrics.accuracy_score(np.asarray(lab_list).ravel(),
                                 np.asarray(prd_list).ravel())
    
    test_loss = np.asarray(test_loss)
    
    # Printing training epoch loss and metrics.
    print('--------------------------------------------------------------------')
    print('[epoch %d], [test loss %.4f +/- %.4f], [acc %.4f]' % (
        epoch, test_loss.mean(), test_loss.std(), acc))
    print('--------------------------------------------------------------------')

# Iterando sobre epochs

In [17]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train(train_loader, net, criterion, optimizer, epoch)

    # Computing test loss and metrics.
    test(test_loader, net, criterion, epoch)

--------------------------------------------------------------------
[epoch 1], [train loss 1.9660 +/- 0.5083]
--------------------------------------------------------------------
--------------------------------------------------------------------
[epoch 1], [test loss 0.7367 +/- 0.0493], [acc 0.7701]
--------------------------------------------------------------------
--------------------------------------------------------------------
[epoch 2], [train loss 0.3951 +/- 0.1424]
--------------------------------------------------------------------
--------------------------------------------------------------------
[epoch 2], [test loss 0.2270 +/- 0.0295], [acc 0.9340]
--------------------------------------------------------------------
--------------------------------------------------------------------
[epoch 3], [train loss 0.1652 +/- 0.0398]
--------------------------------------------------------------------
--------------------------------------------------------------------
[epoc